In [ ]:
import pandas as pd
import numpy as np
import os
openAI_api_key = os.getenv('OPENAI_API_KEY')
if openAI_api_key:
    print("API key found")
else:
    print("API key not found")

In [ ]:
# Attempt to read the uploaded .txt file with an alternative encoding
file_path = 'C:/Users/gabriel.pictet/Documents/Gabriel/REAL/Moti/Moti data/answers-volunteers.txt'
df = pd.read_csv(file_path, sep=",", encoding='ISO-8859-1')
#alternative encoding not working: df = pd.read_csv(file_path, sep=",", encoding='cp1252')
# Display the first few rows of the dataframe to verify successful loading
df.head()


In [3]:
# Open the Raw Moti Data CSV file as sent by CE
#          Volunteers:  answers-volunteers.csv
#          staff: answers-staff.csv  (see below)
# The csv file is corrupted by MS Excel so it needs to be opened in notepad and saved as a .txt file

df = pd.read_csv("C:/Users/gabriel.pictet/Documents/Gabriel/REAL/Moti/Moti data/answers-volunteers.txt", sep=",")

In [6]:
# STAFF: Open the Raw Moti Data CSV file as sent by CE

df = pd.read_csv("C:/Users/gabriel.pictet/Documents/Gabriel/REAL/Moti/Moti data/answers-staff.txt", sep=",")
df.shape

In [ ]:
# STEP 1
# STEP 1.1 Create country variable based on Team Name
# Assigns a country to each team
# Prompts user to allocate country to new teams
# and updates country_team_dictionary Runs allocation
# Use this snippet for VOLUNTEERS and STAFF datasets

with open("team_country_allocation.py") as file:
    exec(file.read())

In [ ]:
   
# STEP 1.2 - dropping illigitimate ballots

#  1.2.1 select and check all the ballots where country = test
dropped_test_teams_df = df[df['country'] == 'test'][['country', 'Team Name']].copy()

# 1.2.2 select and check all the ballots where country = 'Corporate'
dropped_Corporate_teams_df = df[df['country'] == 'Corporate'][['country', 'Team Name']].copy()

print(dropped_test_teams_df)
dropped_test_teams_df.shape
print(dropped_Corporate_teams_df)
dropped_Corporate_teams_df.shape

# 1.2.3      drop illigitimate ballots from test and corporate teams.
df = df.drop(df[df.country == 'test'].index)
df = df.drop(df[df.country == 'Corporate'].index)
df.shape

In [ ]:
# STEP 1.3v : recoding answers from string to integer in order to compute scores and means
# FOR VOLUNTEER DATA
# 1.3.1v Replacing string values with integers to compute scores and means

# Define the mapping of categorical values to integers
mapping = {
    'Strongly disagree': 1,
    'Somewhat disagree': 2,
    'Neutral': 3,
    'Somewhat agree': 4,
    'Strongly agree': 5
}

# For the 24 first columns, transform the categorical column into integers using the mapping

for col in df.columns[:24]:
    df[col] = df[col].replace(mapping)

#replace string values with real numbers 1-7 to compute means for frequency variable, by teams
# At my volunteer activity, I feel strong and vigorous      column[24].      --> positive outcome = 7 so reverse order
# I feel emotionally drained from my work. column[25]      ---> positive outcome = 1
# I feel frustrated by my work.            column[26]      ---> positive outcome = 1

# for volunteers: 1="every time" and for staff: 1="every day" - correct mapping below accordingly.

# At work, I feel strong and vigorous      column[24].      --> positive outcome = 7 so reverse order

mapping = {
    "Every time": 7,
    "Very often": 6,
    "Often": 5,
    "Sometimes": 4,
    "Rarely": 3,
    "Almost never": 2,
    "Never": 1
} 

df['At my volunteer activity, I feel strong and vigorous.'] = df['At my volunteer activity, I feel strong and vigorous.'].replace(mapping)

# 'I feel emotionally drained from my volunteer activity.
# 'I feel frustrated by my volunteer activity.
#mapping = {
#    "Every time": 1,
#    "Very often": 2,
#    "Often": 3,
#    "Sometimes": 4,
#    "Rarely": 5,
#    "Almost never": 6,
#    "Never": 7
#}

df['I feel emotionally drained from my volunteer activity.'] = df['I feel emotionally drained from my volunteer activity.'].replace(mapping)
df['I feel frustrated by my volunteer activity.'] = df['I feel frustrated by my volunteer activity.'].replace(mapping)

df.head()

In [ ]:
# STEP 1.3s : recoding answers from string to integer in order to compute scores and means
# FOR STAFF DATA
# 1.3.1s Replacing string values with integers to compute scores and means

# Define the mapping of categorical values to integers
mapping = {
    'Strongly disagree': 1,
    'Somewhat disagree': 2,
    'Neutral': 3,
    'Somewhat agree': 4,
    'Strongly agree': 5
}

# For the 24 first columns, transform the categorical column into integers using the mapping

for col in df.columns[:24]:
    df[col] = df[col].replace(mapping)

#replace string values with real numbers 1-7 to compute means for frequency variable, by teams
# At work, I feel strong and vigorous      column[24].      --> positive outcome = 7 so reverse order
# I feel emotionally drained from my work. column[25]      ---> positive outcome = 1
# I feel frustrated by my work.            column[26]      ---> positive outcome = 1

# for volunteers: 1="every time" and for staff: 1="every day" - correct mapping below accordingly.

# At work, I feel strong and vigorous      column[24].      --> positive outcome = 7 so reverse order

mapping = {
    "Every day": 7,
    "Very often": 6,
    "Often": 5,
    "Sometimes": 4,
    "Rarely": 3,
    "Almost never": 2,
    "Never": 1
}

df['At work, I feel strong and vigorous.'] = df['At work, I feel strong and vigorous.'].replace(mapping)
df['I feel emotionally drained from my work.'] = df['I feel emotionally drained from my work.'].replace(mapping)
df['I feel frustrated by my work.'] = df['I feel frustrated by my work.'].replace(mapping)

# I feel emotionally drained from my work.
# I feel frustrated by my work.
#mapping = {
#    "Every day": 1,
#    "Very often": 2,
#    "Often": 3,
#    "Sometimes": 4,
#    "Rarely": 5,
#    "Almost never": 6,
#    "Never": 7
#}

#df['I feel emotionally drained from my work.'] = df['I feel emotionally drained from my work.'].replace(mapping)
#df['I feel frustrated by my work.'] = df['I feel frustrated by my work.'].replace(mapping)


In [33]:
# FOR VOLUNTEER DATA
# 1.3.2v We now create the scores for each dimension, e.g. autonomy, belonging, etc.
# NB: Change questions for staff data

#  Before computing the well-being score 
#  we need to recode the two following columns so that the positive outcomes get the higher scores
# 'I feel emotionally drained from my volunteer activity.
# 'I feel frustrated by my volunteer activity.

mapping = {
    7: 1,
    6: 2,
    5: 3,
    4: 4,
    3: 5,
    2: 6,
    1: 7
}

df['I feel emotionally drained from my volunteer activity.'] = df['I feel emotionally drained from my volunteer activity.'].replace(mapping)
df['I feel frustrated by my volunteer activity.'] = df['I feel frustrated by my volunteer activity.'].replace(mapping)

# Specify the column names for Well-being
column_names = [
    'At my volunteer activity, I feel strong and vigorous.',
    'I feel emotionally drained from my volunteer activity.',
    'I feel frustrated by my volunteer activity.'
]

# Calculate the mean row by row and assign it to a new column 'Well-being'
df['Well-being'] = df[column_names].mean(axis=1)

# Rescale the 'Well-being' values from a range of 1-7 to 1-5 and runds the result
df['Well-being'] = (df['Well-being'] - 1) * (4 / 6) + 1
df['Well-being'] = df['Well-being'].round(1)

## recode back to original values for the two "negative" variables
#  after computing the well-being score for "logical" correlation analysis
#  we need to recode the two following columns so that the negative outcomes get the higher scores
# 'I feel emotionally drained from my volunteer activity.
# 'I feel frustrated by my volunteer activity.

mapping = {
    7: 1,
    6: 2,
    5: 3,
    4: 4,
    3: 5,
    2: 6,
    1: 7
}

df['I feel emotionally drained from my volunteer activity.'] = df['I feel emotionally drained from my volunteer activity.'].replace(mapping)
df['I feel frustrated by my volunteer activity.'] = df['I feel frustrated by my volunteer activity.'].replace(mapping)



# Specify the column names for Engagement
column_names = [
    'If I could choose, I will be volunteering one year from now.',
    'The team has a great deal of personal meaning for me.',
    'Considering everything, I am satisfied with my volunteer activity.',
    'I share my ideas with others to improve the team.',
    'The things that I value in life are very similar to the things that the team values.'
]

# Calculate the mean row by row and assign it to a new column 'Engagement'
df['Engagement'] = df[column_names].mean(axis=1).round(1)

# Specify the column names for Autonomy
column_names = [
    'I feel like I can be myself at my volunteer activity.',
    'The tasks I must do at my volunteer activity are in line with what I really want to do.',
    'I am free to express my ideas and opinions on the volunteer activity.'
]

# Calculate the mean row by row and assign it to a new column 'Autonomy'
df['Autonomy'] = df[column_names].mean(axis=1).round(1)

# Specify the column names for belonging
column_names = [
    'At my volunteer activity, I feel part of a group.',
    'At my volunteer activity, I can talk with people about things that really matter to me.',
    'People at my volunteer activity care about me.'
]

# Calculate the mean row by row and assign it to a new column 'Belonging'
df['Belonging'] = df[column_names].mean(axis=1).round(1)

# Specify the column names for Competence
column_names = [
    'I really master my tasks at my volunteer activity.',
    'I feel competent at my volunteer activity.'
]

# Calculate the mean row by row and assign it to a new column 'Competence'
df['Competence'] = df[column_names].mean(axis=1).round(1)



# Specify the column names for Needs
column_names = [
    'I feel like I can be myself at my volunteer activity.',
    'The tasks I must do at my volunteer activity are in line with what I really want to do.',
    'I am free to express my ideas and opinions on the volunteer activity.',
    'At my volunteer activity, I feel part of a group.',
    'At my volunteer activity, I can talk with people about things that really matter to me.',
    'People at my volunteer activity care about me.',
    'I really master my tasks at my volunteer activity.',
    'I feel competent at my volunteer activity.'
]


# Calculate the mean row by row and assign it to a new column 'Needs'
df['Needs'] = df[column_names].mean(axis=1).round(1)


# Specify the column names for Leadership
column_names = [
    'My supervisor listens to how I would like to do things.',
    'I feel understood by my supervisor.',
    'My supervisor encourages me to ask questions.'
]

# Calculate the mean row by row and assign it to a new column 'Leadership'
df['Leadership'] = df[column_names].mean(axis=1).round(1)


# Specify the column names for 'Management'
column_names = [
    'I have the opportunity to develop my social network in my volunteer activity.',
    'I have been able to learn interesting new skills on my volunteer activity.',
    'I am fairly rewarded considering the responsibilities I have.',
    'I am fairly rewarded for the work I do well.',
    'My family, friends and my neighborhood appreciate the work I do for the team.',
    'I feel my work has positive impact on other people.'
]

# Calculate the mean row by row and assign it to a new column 'Management'
df['Management'] = df[column_names].mean(axis=1).round(1)

# Specify the column names for 'Returns'
column_names = [
    'I have the opportunity to develop my social network in my volunteer activity.',
    'I have been able to learn interesting new skills on my volunteer activity.'
]

# Calculate the mean row by row and assign it to a new column 'Returns'
df['Returns'] = df[column_names].mean(axis=1).round(1)

# Specify the column names for 'Rewards'
column_names = [
    'I am fairly rewarded considering the responsibilities I have.',
    'I am fairly rewarded for the work I do well.'
]

# Calculate the mean row by row and assign it to a new column 'Rewards'
df['Rewards'] = df[column_names].mean(axis=1).round(1)

# Specify the column names for 'Status'
column_names = [
    'My family, friends and my neighborhood appreciate the work I do for the team.',
    'I feel my work has positive impact on other people.'
]

# Calculate the mean row by row and assign it to a new column 'Status'
df['Status'] = df[column_names].mean(axis=1).round(1)


In [10]:
# FOR STAFF DATA
# # 1.3.2s We now create the scores for each dimension, e.g. autonomy, belonging, etc.
# NB: Change questions for staff data

#  Before computing the well-being score 
#  we need to recode the two following columns so that the positive outcomes get the higher scores
# 'I feel emotionally drained from my volunteer activity.
# 'I feel frustrated by my volunteer activity.

mapping = {
    7: 1,
    6: 2,
    5: 3,
    4: 4,
    3: 5,
    2: 6,
    1: 7
}

df['I feel emotionally drained from my work.'] = df['I feel emotionally drained from my work.'].replace(mapping)
df['I feel frustrated by my work.'] = df['I feel frustrated by my work.'].replace(mapping)

# Specify the column names for Well-being
column_names = [
    'At work, I feel strong and vigorous.',
    'I feel emotionally drained from my work.',
    'I feel frustrated by my work.'
]

# Calculate the mean row by row and assign it to a new column 'Well-being'
df['Well-being'] = df[column_names].mean(axis=1).round(1)

# Rescale the 'Well-being' values from a range of 1-7 to 1-5 and runds the result
df['Well-being'] = (df['Well-being'] - 1) * (4 / 6) + 1
df['Well-being'] = df['Well-being'].round(1)

## recode back to riginal values for the two "negative" variables
#  after computing the well-being score for "logical" correlation analysis
#  we need to recode the two following columns so that the negative outcomes get the higher scores
# 'I feel emotionally drained from my work.
# 'I feel frustrated by my work.

mapping = {
    7: 1,
    6: 2,
    5: 3,
    4: 4,
    3: 5,
    2: 6,
    1: 7
}

df['I feel emotionally drained from my work.'] = df['I feel emotionally drained from my work.'].replace(mapping)
df['I feel frustrated by my work.'] = df['I feel frustrated by my work.'].replace(mapping)


# Specify the column names for Engagement
column_names = [
    'If I could choose, I would continue working in my team one year from now.',
    'The team means a lot to me personally.',
    'Considering everything, I am satisfied with my work.',
    'I share my ideas with others to improve the team.',
    'The things that I value in life are very similar to the things that the team values.'
]

# Calculate the mean row by row and assign it to a new column 'Engagement'
df['Engagement'] = df[column_names].mean(axis=1).round(1)

# Specify the column names for Autonomy
column_names = [
    'I feel like I can be myself at work.',
    'My tasks at my work are in line with what I really want to do.',
    'I am free to express my ideas and opinions on my work.'
]

# Calculate the mean row by row and assign it to a new column 'Autonomy'
df['Autonomy'] = df[column_names].mean(axis=1).round(1)

# Specify the column names for belonging
column_names = [
    'At my work, I feel part of a group.',
    'At my work, I can talk with people about things that really matter to me.',
    'People at my work care about me.'
]

# Calculate the mean row by row and assign it to a new column 'Belonging'
df['Belonging'] = df[column_names].mean(axis=1).round(1)

# Specify the column names for Competence
column_names = [
    'I really master my tasks at work.',
    'I feel competent at work.'
]

# Calculate the mean row by row and assign it to a new column 'Competence'
df['Competence'] = df[column_names].mean(axis=1).round(1)


# Specify the column names for Needs
column_names = [
    'I feel like I can be myself at work.',
    'My tasks at my work are in line with what I really want to do.',
    'I am free to express my ideas and opinions on my work.',
    'At my work, I feel part of a group.',
    'At my work, I can talk with people about things that really matter to me.',
    'People at my work care about me.',
    'I really master my tasks at work.',
    'I feel competent at work.'
]

# Calculate the mean row by row and assign it to a new column 'Needs'
df['Needs'] = df[column_names].mean(axis=1).round(1)



# Specify the column names for Leadership
column_names = [
    'My supervisor listens to how I would like to do things.',
    'I feel understood by my supervisor.',
    'My supervisor encourages me to ask questions.'
]

# Calculate the mean row by row and assign it to a new column 'Leadership'
df['Leadership'] = df[column_names].mean(axis=1).round(1)

# Specify the column names for 'Management'
column_names = [
    'I have the opportunity to develop my social network at work.',
    'I have been able to learn interesting new skills at work.',
    'I am fairly rewarded considering the responsibilities I have.',
    'I am fairly rewarded for the work I do well.',
    'My family and friends appreciate the work I do for the team.',
    'I feel my work has positive impact on other people.'
]

# Calculate the mean row by row and assign it to a new column 'Management'
df['Management'] = df[column_names].mean(axis=1).round(1)

# Specify the column names for 'Returns'
column_names = [
    'I have the opportunity to develop my social network at work.',
    'I have been able to learn interesting new skills at work.'
]

# Calculate the mean row by row and assign it to a new column 'Returns'
df['Returns'] = df[column_names].mean(axis=1).round(1)

# Specify the column names for 'Rewards'
column_names = [
    'I am fairly rewarded considering the responsibilities I have.',
    'I am fairly rewarded for the work I do well.'
]

# Calculate the mean row by row and assign it to a new column 'Rewards'
df['Rewards'] = df[column_names].mean(axis=1).round(1)

# Specify the column names for 'Status'
column_names = [
    'My family and friends appreciate the work I do for the team.',
    'I feel my work has positive impact on other people.'
]

# Calculate the mean row by row and assign it to a new column 'Status'
df['Status'] = df[column_names].mean(axis=1).round(1)



In [34]:
# STEP 1.4   VOLUNTEERS AND STAFF
# Rename columns in their abridged form to allow VOL-STAFF df concatenation
cols_to_update = [
    'Myself',
    'Tasks',
    'Free',
    'Mastery',
    'Competent',
    'Learn',
    'Group',
    'Talk',
    'Care',
    'Satisfied',
    'Meaning',
    'Stay',
    'Ideas',
    'Values',
    'Understands',
    'Encourages',
    'Listens',
    'Network',
    'Friendly',
    'Team',
    'Appreciated',
    'Responsibilities',
    'Work',
    'Impact',
    'Strong',
    'Drained',
    'Frustrated'
    ] + df.columns[27:].tolist()

df.columns = cols_to_update

In [ ]:
df.info()

In [36]:
# STEP 1.5v FOR VOLUNTEERS
# Export VOLUNTEER dataframe with composite indicators before grouping by teams
ungrouped_df = df
#ungrouped_df.to_excel('ungrouped_volunteers.xlsx', index=False)
ungrouped_df.to_csv('ungrouped_volunteers.csv', index=False)

In [15]:
# STEP 1.5s FOR STAFF
# Export STAFF dataframe with composite indicators before grouping by teams
#from openpyxl import Workbook
#wb.save("sample.xlsx")
ungrouped_df = df
#ungrouped_df.to_excel('ungrouped_staff.xlsx', index=False)
ungrouped_df.to_csv('ungrouped_staff.csv', index=False)

In [41]:
# STEP 2  -  Create one data frame with all ungrouped data with abridged column names:
# Here we merge the two team summary tables, one for Staff / Volunteer comparisons

# Define the columns to include in the merged dataframe
columns = ['Team Name', 'country', 'Well-being', 'Needs', 'Engagement', 'Autonomy', 'Belonging',
           'Competence', 'Leadership', 'Management', 'Returns',
           'Rewards', 'Status',
    'Myself',
    'Tasks',
    'Free',
    'Mastery',
    'Competent',
    'Learn',
    'Group',
    'Talk',
    'Care',
    'Satisfied',
    'Meaning',
    'Stay',
    'Ideas',
    'Values',
    'Understands',
    'Encourages',
    'Listens',
    'Network',
    'Friendly',
    'Team',
    'Appreciated',
    'Responsibilities',
    'Work',
    'Impact',
    'Strong',
    'Drained',
    'Frustrated']

# Read VOLUNTEER dataframe
df_VOL = pd.read_csv("C:/Users/gabriel.pictet/Documents/Gabriel/REAL/Moti/Moti data/ungrouped_volunteers.csv")

# Create the table with the desired columns
df_VOL = df_VOL[columns]

# Round the values to one decimal point
df_VOL = df_VOL.round(decimals=1)

# Create a Volunteer dummy variable where Volunteer = 1
df_VOL['Volunteer'] = 1
df_VOL['Staff'] = 0
df_VOL['Position'] = 'Volunteer'

# Read STAFF dataframe
df_STAFF = pd.read_csv("C:/Users/gabriel.pictet/Documents/Gabriel/REAL/Moti/Moti data/ungrouped_staff.csv")

# Create the table with the desired columns
df_STAFF = df_STAFF[columns]

# Round the values to one decimal point
df_STAFF = df_STAFF.round(decimals=1)

# Create a Staff dummy variable where Staff = 1
df_STAFF['Volunteer'] = 0
df_STAFF['Staff'] = 1
df_STAFF['Position'] = 'Staff'

# Merge the two dataframes
df_VOLandSTAFF = pd.concat([df_VOL, df_STAFF], ignore_index=True)

df_VOLandSTAFF.shape
#df_VOLandSTAFF.to_excel('ungrouped_ALL.xlsx', index=False)
df_VOLandSTAFF.to_csv('ungrouped_ALL.csv', index=False)

In [ ]:
# STEP 3
# STEP 1.1 Create country variable based on Team Name
# Assigns a country to each team
# Prompts user to allocate country to new teams
# and updates country_team_dictionary Runs allocation


with open("team_country_allocation.py") as file:
    exec(file.read())

In [ ]:
# STEP 3: country tables, one for volunteers, one for staff and one for all.
# STEP 3.ALL: country tables, all respondents, volunteers and staff alike.

# 3.1.ALL Group rows by 'Country' and calculate size (count) for each group
df = pd.read_csv("C:/Users/gabriel.pictet/Documents/Gabriel/REAL/Moti/Moti data/ungrouped_ALL.csv")

## for ALL, i.e. no more distinctions between volunteers and staff
country_df = df.groupby('country').size().reset_index(name='size (n)')

# 3.2.ALL Calculate the mean and median for each column
mean_df = df.groupby('country').mean().reset_index()
median_df = df.groupby('country').median().reset_index()

# 3.3.ALL Count unique occurrences of 'Team Name' for each country
n_teams_df = df.groupby('country')['Team Name'].nunique().reset_index(name='nTeams')

# 3.4.ALL Merge the size (count), mean, median, and nTeams DataFrames
country_df = country_df.merge(n_teams_df, on='country').merge(mean_df, on='country', suffixes=('_mean', '_median')).merge(median_df, on='country', suffixes=('_mean', '_median'))

country_df.shape


In [ ]:
# STEP 3.5.ALL  Identify, and check illigitamate survey ballots where team size smaller than 4
# there is a logic issue here - we cannot get rid of teams < 4 after groupby country.
# if we groupby country from the teamsALL dataset then the means and averages will be wrong...

df = country_df

# How many small teams are we talking about?
dropped_small_teams_df = df[df['size (n)'] < 4][['country', 'size (n)']].copy()
print(dropped_small_teams_df)
dropped_small_teams_df.shape

In [39]:
# STEP 3.6.ALL  Drop from illigitamate teams from country dataset
df = df.drop(df[df['size (n)'] < 4].index)
df.shape

(25, 2)

In [40]:
# STEP 3.7.ALL  Export country-level aggregations to Excel
#country_df.to_excel('country_ALL.xlsx', index=False)
country_df.to_csv('country_ALL.csv', index=False)

In [36]:
# STEP 3: country tables, one for volunteers, one for staff and one for all.
# STEP 3.VOL: country tables volunteers only

# 3.1.VOL Group rows by 'Country' and calculate size (count) for each group
df = pd.read_csv("C:/Users/gabriel.pictet/Documents/Gabriel/REAL/Moti/Moti data/ungrouped_ALL.csv")

## for VOL, i.e.  volunteers only
# Filter the dataframe for 'Position' = 'Volunteer'
df = df[df['Position'] == 'Volunteer']
country_df = df.groupby('country').size().reset_index(name='size (n)')

# 3.2.VOL Calculate the mean and median for each column
mean_df = df.groupby('country').mean().reset_index()
median_df = df.groupby('country').median().reset_index()

# 3.3.VOL Count unique occurrences of 'Team Name' for each country
n_teams_df = df.groupby('country')['Team Name'].nunique().reset_index(name='nTeams')

# 3.4.ALL Merge the size (count), mean, median, and nTeams DataFrames
country_df = country_df.merge(n_teams_df, on='country').merge(mean_df, on='country', suffixes=('_mean', '_median')).merge(median_df, on='country', suffixes=('_mean', '_median'))

country_df.shape

# STEP 3.5.VOL  Identify, and check illigitamate survey ballots where team size smaller than 4
# there is a logic issue here - we cannot get rid of teams < 4 after groupby country.
# if we groupby country from the teamsALL dataset then the means and averages will be wrong...

df = country_df

# How many small teams are we talking about?
dropped_small_teams_df = df[df['size (n)'] < 4][['country', 'size (n)']].copy()
print(dropped_small_teams_df)
dropped_small_teams_df.shape

# STEP 3.6.VOL  Drop from illigitamate teams from country dataset
df = df.drop(df[df['size (n)'] < 4].index)
df.shape

# STEP 3.7.VOL  Export country-level aggregations to Excel
country_df.to_excel('country_VOL.xlsx', index=False)
country_df.to_csv('country_VOL.csv', index=False)


      country  size (n)
8      Global         2
11  Indonesia         1
18     Serbia         1


C:\Users\gabriel.pictet\AppData\Local\Temp\ipykernel_21140\3162689540.py:13: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  mean_df = df.groupby('country').mean().reset_index()
C:\Users\gabriel.pictet\AppData\Local\Temp\ipykernel_21140\3162689540.py:14: FutureWarning: The default value of numeric_only in DataFrameGroupBy.median is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  median_df = df.groupby('country').median().reset_index()


In [37]:
# STEP 3: country tables, one for volunteers, one for staff and one for all.
# STEP 3.STAFF: country tables staff only

# 3.1.STAFF Group rows by 'Country' and calculate size (count) for each group
df = pd.read_csv("C:/Users/gabriel.pictet/Documents/Gabriel/REAL/Moti/Moti data/ungrouped_ALL.csv")

# Filter the dataframe for 'Position' = 'Staff'
df = df[df['Position'] == 'Staff']
country_df = df.groupby('country').size().reset_index(name='size (n)')

# 3.2.STAFF Calculate the mean and median for each column
mean_df = df.groupby('country').mean().reset_index()
median_df = df.groupby('country').median().reset_index()

# 3.3.VOL Count unique occurrences of 'Team Name' for each country
n_teams_df = df.groupby('country')['Team Name'].nunique().reset_index(name='nTeams')

# 3.4.ALL Merge the size (count), mean, median, and nTeams DataFrames
country_df = country_df.merge(n_teams_df, on='country').merge(mean_df, on='country', suffixes=('_mean', '_median')).merge(median_df, on='country', suffixes=('_mean', '_median'))

country_df.shape

# STEP 3.5.STAFF  Identify, and check illigitamate survey ballots where team size smaller than 4
# there is a logic issue here - we cannot get rid of teams < 4 after groupby country.
# if we groupby country from the teamsALL dataset then the means and averages will be wrong...

df = country_df

# How many small teams are we talking about?
dropped_small_teams_df = df[df['size (n)'] < 4][['country', 'size (n)']].copy()
print(dropped_small_teams_df)
dropped_small_teams_df.shape

# STEP 3.6.STAFF  Drop from illigitamate teams from country dataset
df = df.drop(df[df['size (n)'] < 4].index)
df.shape

# STEP 3.7.STAFF  Export country-level aggregations to Excel
country_df.to_excel('country_STAFF.xlsx', index=False)
country_df.to_csv('country_STAFF.csv', index=False)


Empty DataFrame
Columns: [country, size (n)]
Index: []


C:\Users\gabriel.pictet\AppData\Local\Temp\ipykernel_21140\2991092071.py:12: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  mean_df = df.groupby('country').mean().reset_index()
C:\Users\gabriel.pictet\AppData\Local\Temp\ipykernel_21140\2991092071.py:13: FutureWarning: The default value of numeric_only in DataFrameGroupBy.median is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  median_df = df.groupby('country').median().reset_index()


TypeError: agg function failed [how->mean,dtype->object]

Dictionary saved to team_country_dictionary.txt


In [54]:
# STEP 4: create dataframe where one row is one team

# STEP 4.1 Group rows by 'Team Name' and calculate size (count) for each team
df = pd.read_csv("C:/Users/gabriel.pictet/Documents/Gabriel/REAL/Moti/Moti data/ungrouped_ALL.csv")
team_df = df.groupby('Team Name').size().reset_index(name='size (n)')

# STEP 4.2  Calculate the mean and median for each column
#mean_df = df.groupby('Team Name').mean().reset_index()
#median_df = df.groupby('Team Name').median().reset_index()

# STEP 4.3  Merge the size (count), mean and country DataFrames
#team_df = team_df.merge(mean_df, on='Team Name').merge(df[['Team Name', 'country', 'Position']].drop_duplicates(), on='Team Name')

# STEP 4.3  Merge the size (count), mean, median, and country DataFrames
#team_df = team_df.merge(mean_df, on='Team Name', suffixes=('_mean', '_median')).merge(median_df, on='Team Name', suffixes=('_mean', '_median')).merge(df[['Team Name', 'country', 'Position']].drop_duplicates(), on='Team Name')

team_df.info()

# STEP 4.4  
# For ALL (Volunteers and staff) - create files with Teams data
#team_df.to_excel('team_ALL.xlsx', index=False)
team_df.to_csv('team_ALL_short.csv', index=False)
